In [97]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans

# load BLM data file
# stored in a DataFrame variable
blmData = pd.read_csv('data/Fill_6953_start_ramp.txt', sep=' ', header=None)

In [98]:
# Delete first column (contains timestamps)
blmData = blmData.drop(columns=0)

In [100]:
# Get row with max BLM value in the whole table

blmMax = blmData.max() # Series: column num (i.e. BLM num) -> maximum value for that column
blmMaxOverall = blmData.values.max() # max BLM value in the whole dataset
rowIndexWithBlmMaxOverall = blmData.idxmax()[blmMax[blmMax == blmMaxOverall].index]

rowWithBlmMaxOverall = blmData.loc[rowIndexWithBlmMaxOverall]

#TODO what if more than one row contains the same blmMaxOverall?